# Movie Recommendation

## Call Data

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
movies = pd.read_csv('C:/Users/maste/Documents/대학교/대학교 3학년 2학기/기계학습/Final Project/tmdb_5000_movies.csv')
credits = pd.read_csv('C:/Users/maste/Documents/대학교/대학교 3학년 2학기/기계학습/Final Project/tmdb_5000_credits.csv')

## Data Cleaning

In [3]:
for index, row in movies.iterrows():
    genre_list = []
    for genre in json.loads(row['genres']):
        genre_list.append(genre['name'])
    movies.at[index, 'genres'] = str(genre_list)

In [4]:
for index, row in movies.iterrows():
    keyword_list = []
    for keyword in json.loads(row['keywords']):
        keyword_list.append(keyword['name'])
    movies.at[index, 'keywords'] = str(keyword_list)

In [5]:
for index, row in credits.iterrows():
    cast_list = []
    for cast in json.loads(row['cast']):
        cast_list.append(cast['name'])
    credits.at[index, 'cast'] = str(cast_list)

In [6]:
for index, row in credits.iterrows():
    director_list = []
    for crew in json.loads(row['crew']):
        if crew['job'] == 'Director':
            director_list.append(crew['name'])
    credits.at[index, 'director'] = str(director_list)

credits.drop(columns='crew', inplace=True)

In [7]:
movies = movies.merge(credits,left_on='id',right_on='movie_id',how='left')
movies = movies[['id','original_title','genres','cast','vote_average','director','keywords']]
movies.head()

,id,original_title,genres,cast,vote_average,director,keywords
0,19995,Avatar,"['Action', 'Adventure', 'Fantasy', 'Science Fi...","['Sam Worthington', 'Zoe Saldana', 'Sigourney ...",7.2,['James Cameron'],"['culture clash', 'future', 'space war', 'spac..."
1,285,Pirates of the Caribbean: At World's End,"['Adventure', 'Fantasy', 'Action']","['Johnny Depp', 'Orlando Bloom', 'Keira Knight...",6.9,['Gore Verbinski'],"['ocean', 'drug abuse', 'exotic island', 'east..."
2,206647,Spectre,"['Action', 'Adventure', 'Crime']","['Daniel Craig', 'Christoph Waltz', 'Léa Seydo...",6.3,['Sam Mendes'],"['spy', 'based on novel', 'secret agent', 'seq..."
3,49026,The Dark Knight Rises,"['Action', 'Crime', 'Drama', 'Thriller']","['Christian Bale', 'Michael Caine', 'Gary Oldm...",7.6,['Christopher Nolan'],"['dc comics', 'crime fighter', 'terrorist', 's..."
4,49529,John Carter,"['Action', 'Adventure', 'Science Fiction']","['Taylor Kitsch', 'Lynn Collins', 'Samantha Mo...",6.1,['Andrew Stanton'],"['based on novel', 'mars', 'medallion', 'space..."


In [8]:
new_id = list(range(0,movies.shape[0]))
movies['new_id'] = new_id
movies = movies[['new_id','original_title','genres','cast','vote_average','director','keywords']]
movies.head()

,new_id,original_title,genres,cast,vote_average,director,keywords
0,0,Avatar,"['Action', 'Adventure', 'Fantasy', 'Science Fi...","['Sam Worthington', 'Zoe Saldana', 'Sigourney ...",7.2,['James Cameron'],"['culture clash', 'future', 'space war', 'spac..."
1,1,Pirates of the Caribbean: At World's End,"['Adventure', 'Fantasy', 'Action']","['Johnny Depp', 'Orlando Bloom', 'Keira Knight...",6.9,['Gore Verbinski'],"['ocean', 'drug abuse', 'exotic island', 'east..."
2,2,Spectre,"['Action', 'Adventure', 'Crime']","['Daniel Craig', 'Christoph Waltz', 'Léa Seydo...",6.3,['Sam Mendes'],"['spy', 'based on novel', 'secret agent', 'seq..."
3,3,The Dark Knight Rises,"['Action', 'Crime', 'Drama', 'Thriller']","['Christian Bale', 'Michael Caine', 'Gary Oldm...",7.6,['Christopher Nolan'],"['dc comics', 'crime fighter', 'terrorist', 's..."
4,4,John Carter,"['Action', 'Adventure', 'Science Fiction']","['Taylor Kitsch', 'Lynn Collins', 'Samantha Mo...",6.1,['Andrew Stanton'],"['based on novel', 'mars', 'medallion', 'space..."


### Genre Column

In [9]:
movies['genres'] = movies['genres'].str.strip('[]').str.replace(' ','').str.replace("'",'')
movies['genres'] = movies['genres'].str.split(',')

In [10]:
def sort_and_join(genre_list):
    return ', '.join(sorted(genre_list))

movies['genres'] = movies['genres'].apply(sort_and_join)

In [11]:
genre_list = []

for genres in movies['genres']:
    for genre in genres.split(', '):
        if genre not in genre_list and genre != '':
            genre_list.append(genre)

def binary(movie_genres, genre_list):
    binary_list = []

    for genre in genre_list:
        if genre in movie_genres:
            binary_list.append(1)
        else:
            binary_list.append(0)

    return binary_list

In [12]:
movies['binary_genres'] = movies['genres'].apply(binary, genre_list=genre_list)
movies['binary_genres'].head()

0    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: binary_genres, dtype: object

### Director Column

In [13]:
def remove_empty(i):
    if i is None:
        return ''
    return str(i)
movies['director'] = movies['director'].apply(remove_empty)

In [14]:
director_list = []

for i in movies['director']:
    if i not in director_list:
        director_list.append(i)

def binary(director, director_list):
    binary_list = []
    for d in director_list:
        if d in director:
            binary_list.append(1)
        else:
            binary_list.append(0)
    return binary_list

In [15]:
movies['binary_director'] = movies['director'].apply(binary, director_list=director_list)
movies['binary_director'].head()

0    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: binary_director, dtype: object

### Cast Column

In [16]:
movies['cast'] = movies['cast'].str.strip('[]').str.replace(' ','').str.replace("'",'').str.replace('"','')
movies['cast'] = movies['cast'].str.split(',')

In [17]:
cast_list = []

for index, row in movies.iterrows():
    cast = row["cast"][:10]
    
    for i in cast:
        if i not in cast_list:
            cast_list.append(i)

def binary(cast):
    binary_list = []
    
    for actor in cast_list:
        if actor in cast:
            binary_list.append(1)
        else:
            binary_list.append(0)
    
    return binary_list

In [18]:
movies['binary_cast'] = movies['cast'].apply(binary)
movies['binary_cast'].head()

0    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...
1    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...
2    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: binary_cast, dtype: object

### Keyword Column

In [19]:
movies['keywords'] = movies['keywords'].str.strip('[]').str.replace(' ', '').str.replace("'", '').str.replace('"', '')
movies['keywords'] = movies['keywords'].str.split(',')

In [20]:
words_list = []

for index, row in movies.iterrows():
    genres = row["keywords"]
    
    for genre in genres:
        if genre not in words_list:
            words_list.append(genre)

def binary(keywords):
    binary_list = []
    for word in words_list:
        if word in keywords:
            binary_list.append(1)
        else:
            binary_list.append(0)
    return binary_list

In [21]:
movies['binary_words'] = movies['keywords'].apply(binary)
movies['binary_words'].head()

0    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
1    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...
Name: binary_words, dtype: object

In [22]:
movies.head()

,new_id,original_title,genres,cast,vote_average,director,keywords,binary_genres,binary_director,binary_cast,binary_words
0,0,Avatar,"Action, Adventure, Fantasy, ScienceFiction","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",7.2,['James Cameron'],"[cultureclash, future, spacewar, spacecolony, ...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,1,Pirates of the Caribbean: At World's End,"Action, Adventure, Fantasy","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",6.9,['Gore Verbinski'],"[ocean, drugabuse, exoticisland, eastindiatrad...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,Spectre,"Action, Adventure, Crime","[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",6.3,['Sam Mendes'],"[spy, basedonnovel, secretagent, sequel, mi6, ...","[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,The Dark Knight Rises,"Action, Crime, Drama, Thriller","[ChristianBale, MichaelCaine, GaryOldman, Anne...",7.6,['Christopher Nolan'],"[dccomics, crimefighter, terrorist, secretiden...","[1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,John Carter,"Action, Adventure, ScienceFiction","[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",6.1,['Andrew Stanton'],"[basedonnovel, mars, medallion, spacetravel, p...","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."


In [23]:
movies = movies[(movies['vote_average']!=0)]
movies = movies[movies['director']!='']

In [24]:
movies['new_id']= range(movies.shape[0])
movies.head()

,new_id,original_title,genres,cast,vote_average,director,keywords,binary_genres,binary_director,binary_cast,binary_words
0,0,Avatar,"Action, Adventure, Fantasy, ScienceFiction","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",7.2,['James Cameron'],"[cultureclash, future, spacewar, spacecolony, ...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,1,Pirates of the Caribbean: At World's End,"Action, Adventure, Fantasy","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",6.9,['Gore Verbinski'],"[ocean, drugabuse, exoticisland, eastindiatrad...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,Spectre,"Action, Adventure, Crime","[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",6.3,['Sam Mendes'],"[spy, basedonnovel, secretagent, sequel, mi6, ...","[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,The Dark Knight Rises,"Action, Crime, Drama, Thriller","[ChristianBale, MichaelCaine, GaryOldman, Anne...",7.6,['Christopher Nolan'],"[dccomics, crimefighter, terrorist, secretiden...","[1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,John Carter,"Action, Adventure, ScienceFiction","[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",6.1,['Andrew Stanton'],"[basedonnovel, mars, medallion, spacetravel, p...","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."


## Similarity Between Movies

### Euclidean Distance

In [25]:
from scipy import spatial

def distance(movieId1, movieId2):
    a = movies.iloc[movieId1]
    b = movies.iloc[movieId2]
    
    genresA = a['binary_genres']
    genresB = b['binary_genres']
    
    genredistance = spatial.distance.euclidean(genresA, genresB)
    
    castA = a['binary_cast']
    castB = b['binary_cast']
    castdistance = spatial.distance.euclidean(castA, castB)
    
    directA = a['binary_director']
    directB = b['binary_director']
    directdistance = spatial.distance.euclidean(directA, directB)
    
    wordsA = a['binary_words']
    wordsB = b['binary_words']
    wordsdistance = spatial.distance.euclidean(directA, directB)
    
    return genredistance + directdistance + castdistance + wordsdistance

In [26]:
distance(3, 50)

12.959062615397976

In [27]:
print(movies.iloc[3])
print(movies.iloc[50])

new_id                                                             3
original_title                                 The Dark Knight Rises
genres                                Action, Crime, Drama, Thriller
cast               [ChristianBale, MichaelCaine, GaryOldman, Anne...
vote_average                                                     7.6
director                                       ['Christopher Nolan']
keywords           [dccomics, crimefighter, terrorist, secretiden...
binary_genres      [1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
binary_director    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
binary_cast        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
binary_words       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: 3, dtype: object
new_id                                                            50
original_title                   Prince of Persia: The Sands of Time
genres                           Action, Adventure, Fantasy, Romance
cast       

### Recommend Movie

In [28]:
import operator

def calculate_similarity(movie1_id, movie2_id):
    return distance(movie1_id, movie2_id)

def get_neighbors(base_movie, k):
    distances = []

    for index, movie in movies.iterrows():
        if movie['new_id'] != base_movie['new_id'].values[0].astype(int):
            dist = calculate_similarity(base_movie['new_id'].values[0], movie['new_id'])
            distances.append((movie['new_id'], dist))

    distances.sort(key=operator.itemgetter(1))
    return distances[:k]

def recommend_movie(movie_name, k=10):
    new_movie = movies[movies['original_title'].str.contains(movie_name)].iloc[0].to_frame().T
    
    print('Selected Movie: ', new_movie.original_title.values[0])
    
    neighbors = get_neighbors(new_movie, k)
    
    print('\nRecommended Movies: \n')
    for neighbor in neighbors:
        movie_info = movies.iloc[neighbor[0]]
        print(f"{movie_info['original_title']} | Genres: {str(movie_info['genres']).strip('[]').replace(' ', '')} | Rating: {movie_info['vote_average']}")

In [29]:
recommend_movie('The Dark Knight')

Selected Movie:  The Dark Knight Rises

Recommended Movies: 

The Dark Knight | Genres: Action,Crime,Drama,Thriller | Rating: 8.2
Batman Begins | Genres: Action,Crime,Drama | Rating: 7.5
Memento | Genres: Mystery,Thriller | Rating: 8.1
Inception | Genres: Action,Adventure,Mystery,ScienceFiction,Thriller | Rating: 8.1
Insomnia | Genres: Crime,Mystery,Thriller | Rating: 6.8
The Prestige | Genres: Drama,Mystery,Thriller | Rating: 8.0
Interstellar | Genres: Adventure,Drama,ScienceFiction | Rating: 8.1
Jimmy and Judy | Genres: Action,Crime,Drama,Thriller | Rating: 5.4
The Hunted | Genres: Action,Crime,Drama,Thriller | Rating: 6.0
Death Sentence | Genres: Action,Crime,Drama,Thriller | Rating: 6.5


In [30]:
recommend_movie('Interstellar')

Selected Movie:  Interstellar

Recommended Movies: 

Memento | Genres: Mystery,Thriller | Rating: 8.1
Inception | Genres: Action,Adventure,Mystery,ScienceFiction,Thriller | Rating: 8.1
Silent Running | Genres: Adventure,Drama,ScienceFiction | Rating: 6.3
The Prestige | Genres: Drama,Mystery,Thriller | Rating: 8.0
Insomnia | Genres: Crime,Mystery,Thriller | Rating: 6.8
20,000 Leagues Under the Sea | Genres: Adventure,Drama,ScienceFiction | Rating: 6.8
Gerry | Genres: Adventure,Drama,Mystery | Rating: 6.2
The Martian | Genres: Adventure,Drama,ScienceFiction | Rating: 7.6
Midnight Special | Genres: Adventure,Drama,ScienceFiction | Rating: 6.2
End of the Spear | Genres: Adventure,Drama | Rating: 6.1
